### Ce notebook présente la création d'un modèle de caméra géoréférencée pour la géolocalisation de points d'intérêt dans l'image. Il réalise également un benchmark du runtime de la projection de multiple points sur des MNT de taille variée, à l'aide de méthodes séquentielles et vectorielles.

In [1]:
%matplotlib widget
import geopandas as gpd
import logging
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import pyvista as pv

from georefcam.camera_model import CTCameraModel
from georefcam.dem import ASCIIGridDEM
from georefcam.georefcam import GeoRefCam
from pathlib import Path
from PIL import Image
from time import time

# logging.basicConfig(level=logging.DEBUG)

# Définition des dossiers contenant les données nécessaires à l'exécution du notebook (À ÉDITER)

In [2]:
carto_dir = Path("/home/florent/git_repos/d4g/pyronear/explo/cartos/BDALTIV2_2-0_25M_ASC_LAMB93-IGN69_D007_2022-12-16/BDALTIV2")
camdata_dir = Path("/home/florent/git_repos/d4g/pyronear/explo/pyronear_cam_data")
donnees_dir = Path("1_DONNEES_LIVRAISON_2023-01-00224/BDALTIV2_MNT_25M_ASC_LAMB93_IGN69_D007")
dalles_dir = Path("3_SUPPLEMENTS_LIVRAISON_2023-01-00224/BDALTIV2_MNT_25M_ASC_LAMB93_IGN69_D007/")

cam_name = "brison_4"
test_img_name = "pyronear_brison_4_2023_07_04T06_07_57.jpg"
gcp_filepath = "df_annotations.pkl"

# Récupération des données de localisation/orientation de la caméra

Création d'un geodataframe des dalles dans la carto contenant une caméra (problème si les dalles recherchées sont à cheval sur deux cartos, cas relativement improbable d'une caméra couvrant la bordure entre deux départements) :

In [3]:
dalles_shape_file = Path("dalles.shp")
gdf_dalles = gpd.read_file(carto_dir / dalles_dir / dalles_shape_file)

Création d'un geodataframe des caméras utilisées à l'aide du fichier de référence 'API_DATA - devices.csv' :

In [4]:
cam_file = camdata_dir / "API_DATA - devices.csv"
df_cams = pd.read_csv(cam_file)
gdf_cams = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_cams.lon, df_cams.lat, crs="WGS84"), data=df_cams)

Sélection d'une caméra et récupération de ses informations, sélection du fichier de dalle contenant sa localisation

In [5]:
cam_info = gdf_cams[gdf_cams.login == cam_name].squeeze()
test_img_path = camdata_dir / cam_name / test_img_name
test_img = Image.open(test_img_path)
test_img_res = test_img.size

gdf_dalles_wgs84 = gdf_dalles.to_crs("WGS84")
dalle_cam = gdf_dalles[gdf_dalles_wgs84.geometry.contains(cam_info.geometry)].squeeze()
nom_dalle = dalle_cam.NOM_DALLE
dalle_file = carto_dir / donnees_dir / Path(f"{nom_dalle}.asc")

# Création d'une caméra géoréférencée avec `georefcam`

Création d'un DEM avec `ASCIIGridDEM`

In [8]:
dem = ASCIIGridDEM(dalle_file, gdf_dalles.crs)
dem.build_pcd(sample_step=5)
dem.build_mesh()

Création d'un modèle de caméra orientée et localisée avec `CTCameraModel`

In [7]:
cam_view_x_angle, cam_view_y_angle = 87, 44
cam_roll = 0
cam_model = SimpleCameraModel(
    test_img_res,
    cam_view_x_angle,
    cam_view_y_angle,
    cam_info.azimuth,
    cam_info.pitch,
    cam_roll,
    cam_info.lat,
    cam_info.lon,
    cam_info.elevation,
    dem.crs
)

Création du modèle de caméra géoréférencée

In [9]:
geocam = GeoRefCam(cam_model, dem)

# Évaluation de la projection d'un ensemble de GCP

In [6]:
with open(gcp_filepath, "rb") as f:
    df_annot_gcp = pickle.load(f)
df_annot_gcp = df_annot_gcp.rename({"ele": "alt"}, axis=1).astype({"posx": int, "posy": int, "lat": float, "lon": float, "alt": float})
df_annot_gcp
gdf_annot_gcp = gpd.GeoDataFrame(df_annot_gcp, geometry=gpd.points_from_xy(df_annot_gcp.lon, df_annot_gcp.lat, df_annot_gcp.alt), crs="WGS84")
gdf_annot_gcp

,posx,posy,lat,lon,alt,geometry
0,651,328,44.609620,4.098371,1448.0,POINT Z (4.09837 44.60962 1448.00000)
1,789,319,44.612899,4.126470,1458.0,POINT Z (4.12647 44.61290 1458.00000)
2,978,409,44.577702,4.188103,643.0,POINT Z (4.18810 44.57770 643.00000)
3,1190,310,44.612280,4.185187,1316.0,POINT Z (4.18519 44.61228 1316.00000)


Sélection d'un ensemble de GCPs et génération de leurs rayons de projection dans le repère monde

In [10]:
test_gcp_idx = [2, 2]
pixel_points = gdf_annot_gcp.loc[test_gcp_idx, ["posx", "posy"]]
# pixel_points = gdf_annot_gcp[["posx", "posy"]]

rays = geocam.camera_model.project_pixel_points_to_world_rays(pixel_points)
rays

array([[[ 44.54515457,  44.67704565],
        [  4.21653414,   4.14014126],
        [780.        ,   0.        ]],

       [[ 44.54515457,  44.67704565],
        [  4.21653414,   4.14014126],
        [780.        ,   0.        ]]])

Benchmarks de runtime de la projection de rayons séquentielle et vectorielle, pour des collections de rayons et tailles de DEM 

In [11]:
exp_cast_m = ["vec", "seq"]
exp_n_rays = [10]
exp_dem_sample_steps = [10, 5]
exp_max_runtime_s = 600
exp_results = []

for sample_step in exp_dem_sample_steps:
    
    dem = ASCIIGridDEM(dalle_file, gdf_dalles.crs)
    dem.build_pcd(sample_step=sample_step)
    dem.build_mesh()
    geocam = GeoRefCam(cam_model, dem)
    
    for n_rays in exp_n_rays:
        test_gcp_idx = [2] * n_rays
        pixel_points = gdf_annot_gcp.loc[test_gcp_idx, ["posx", "posy"]]
        pixel_points.posx += np.linspace(0, test_img_res[0], num=n_rays).astype(int)
        rays = geocam.camera_model.project_pixel_points_to_world_rays(pixel_points)
        n_points = int(geocam.dem.pcd.size / 3)
        
        for cast_m in exp_cast_m:    
            print(f"cast_m = {cast_m} | n_rays = {n_rays} | DEM points: {n_points}")
            start = time()
            filtered_inter_points, filtered_inter_triangles, df_ray = geocam.cast_rays(
                rays, check_crs=True, min_d_intersection_m=0, return_df_ray=True, cast_method=cast_m, max_cp_time_s=exp_max_runtime_s)
            stop = time()

            runtime = stop - start
            timeout = df_ray is None
            exp_results.append([cast_m, n_rays, n_points, stop - start, timeout, df_ray])

            print(f"Cast time: {stop - start:.2f} s | Timeout: {timeout}\n\n")

cast_m = vec | n_rays = 10 | DEM points: 10000
Cast time: 0.52 s | Timeout: False


cast_m = seq | n_rays = 10 | DEM points: 10000
Cast time: 0.12 s | Timeout: False


cast_m = vec | n_rays = 10 | DEM points: 40000
Cast time: 0.59 s | Timeout: False


cast_m = seq | n_rays = 10 | DEM points: 40000
Cast time: 0.20 s | Timeout: False




In [18]:
pd.testing.assert_frame_equal(exp_results[0][5], exp_results[1][5])

AssertionError: DataFrame.iloc[:, 6] (column name="inter_x") are different

DataFrame.iloc[:, 6] (column name="inter_x") values are different (100.0 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[left]:  [797136.4368621425, 797137.5802774458, 797138.7233165373, 797139.8659798873, 797141.0082679661, 797142.1582215589, 797143.2997578719, 797144.440920328, 797145.5817093974, 797146.730155329]
[right]: [794729.3125, 795650.3125, 796150.1875, 796547.875, 796852.75, 797106.375, 797351.6875, 797598.625, 797787.1875, 797957.9375]

# TODO: VISUALISER ET COMPARER LES PROJS DE RAYONS GÉNÉRÉS PAR LES MÉTHODES SEQ ET VEC

In [12]:
# import numpy as np

# plot_margin_m = 1000

# target_points = np.array([list(gdf_annot_gcp.to_crs(gdf_dalles.crs).loc[idx, "geometry"].coords)[0] for idx in test_gcp_idx])
# origins = df_ray[["ori_x", "ori_y", "ori_z"]].values
# inter_points = df_ray[["inter_x", "inter_y", "inter_z"]].values

# print(f"target_points:\n{target_points}\n\norigins:\n{origins}\n\ninter_points:\n{inter_points}")

# points_of_interest = np.vstack((target_points, origins, inter_points))
# (x_min, y_min, _) = points_of_interest.min(axis=0) - plot_margin_m
# (x_max, y_max, _) = points_of_interest.max(axis=0) + plot_margin_m

# alts_meshgrid = dem.pcd
# zone_mask_idx = np.vstack(((alts_meshgrid[:, :, 0] > x_min) & (alts_meshgrid[:, :, 0] < x_max) & (alts_meshgrid[:, :, 1] > y_min) & (alts_meshgrid[:, :, 1] < y_max)).nonzero())
# x_min_idx, y_min_idx = zone_mask_idx.min(axis=1)
# x_max_idx, y_max_idx = zone_mask_idx.max(axis=1)
# plot_meshgrid = alts_meshgrid[x_min_idx:x_max_idx, y_min_idx:y_max_idx, :]

# plot_pv_meshgrid = pv.StructuredGrid(*[plot_meshgrid[:, :, i] for i in range(3)])
# plot_pv_meshgrid["alt"] = plot_meshgrid[:, :, 2].ravel(order="F")  # add the altitude as a scalar field in order to plot it as a colormap


# plotter = pv.Plotter(lighting="none")
# plotter.add_mesh(plot_pv_meshgrid, smooth_shading=True, specular=0.5, specular_power=15)
# light = pv.Light()
# light.set_direction_angle(45, -20)
# plotter.add_light(light)

# bar_height_m = round(max(abs(alts_meshgrid[0, x_max_idx, 0] - alts_meshgrid[0, x_min_idx, 0]), abs(alts_meshgrid[y_max_idx, 0, 1] - alts_meshgrid[y_min_idx, 0, 1])) / 10)

# targets_pl = [pv.Line(target_point, target_point + np.array([0, 0, bar_height_m])) for target_point in target_points]
# origins_pl = [pv.Line(origin, origin + np.array([0, 0, bar_height_m])) for origin in origins]
# inters_pl = [pv.Line(inter_point, inter_point + np.array([0, 0, bar_height_m])) for inter_point in inter_points]
# ray_cuts = [pv.Line(origin, inter_point) for origin, inter_point in zip(origins, inter_points)]
# sep_lines = [pv.Line([target_point[0], target_point[1], max(target_point[2], inter_point[2])], [inter_point[0], inter_point[1], max(target_point[2], inter_point[2])]) for target_point, inter_point in zip(target_points, inter_points)]
# intersections = [pv.PolyData(inter_point) for inter_point in inter_points]

# for n_proj in range(len(target_points)):
#     plotter.add_mesh(origins_pl[n_proj], color="red", line_width=2, opacity=1)
#     plotter.add_mesh(ray_cuts[n_proj], color="orange", line_width=2, opacity=1)
#     plotter.add_mesh(inters_pl[n_proj], color="lightgreen", line_width=2, opacity=1)
#     plotter.add_mesh(targets_pl[n_proj], color="yellow", line_width=2, opacity=1)
#     plotter.add_mesh(sep_lines[n_proj], color="cyan", line_width=2, opacity=1,)
#     plotter.add_mesh(intersections[n_proj], color="maroon", point_size=5)

# plotter.show()

# Benchmarks de runtime de la projection de rayons séquentielle et vectorielle, pour des collections de rayons et tailles de DEM croissantes

Nous constatons que pour des collections de moins de 1000 rayons à projeter, la méthode séquentielle est la plus rapide. Au-delà de ce nombre, la méthode vectorielle devient plus efficace. La taille du MNT n'influence pas la position de cette délimitation, mais elle influe sur le gain de temps apporté par la méthode (plus le MNT contient de points, plus la différence de temps de calcul entre les deux méthodes est importante).

EXPERIMENTAL RESULTS

-------------------------------------------------------

PARAMETER SPACE:

```
exp_cast_m = ["vec", "seq"]
exp_n_rays = [1, 10, 100, 1000, 10000]
exp_dem_sample_steps = [10, 5, 2, 1]
exp_max_runtime_s = 350
```

-------------------------------------------------------

SETUP:

```
Architecture :                              x86_64
  Mode(s) opératoire(s) des processeurs :   32-bit, 64-bit
  Address sizes:                            48 bits physical, 48 bits virtual
  Boutisme :                                Little Endian
Processeur(s) :                             16
  Liste de processeur(s) en ligne :         0-15
Identifiant constructeur :                  AuthenticAMD
  Nom de modèle :                           AMD Ryzen 7 5700U with Radeon Graphics
    Famille de processeur :                 23
    Modèle :                                104
    Thread(s) par cœur :                    2
    Cœur(s) par socket :                    8
    Socket(s) :                             1
    Révision :                              1
    Vitesse maximale du processeur en MHz : 4372,0000
    Vitesse minimale du processeur en MHz : 400,0000
    BogoMIPS :                              3592.98

MiB Mem :  15325,9 total
```

-------------------------------------------------------

RESULTS:

```
cast_m = vec | n_rays = 1 | DEM points: 10000
Cast time: 0.57 s | Timeout: False

cast_m = seq | n_rays = 1 | DEM points: 10000
Cast time: 0.13 s | Timeout: False

cast_m = vec | n_rays = 10 | DEM points: 10000
Cast time: 0.21 s | Timeout: False

cast_m = seq | n_rays = 10 | DEM points: 10000
Cast time: 0.11 s | Timeout: False

cast_m = vec | n_rays = 100 | DEM points: 10000
Cast time: 0.23 s | Timeout: False

cast_m = seq | n_rays = 100 | DEM points: 10000
Cast time: 0.14 s | Timeout: False

cast_m = vec | n_rays = 1000 | DEM points: 10000
Cast time: 0.49 s | Timeout: False

cast_m = seq | n_rays = 1000 | DEM points: 10000
Cast time: 0.40 s | Timeout: False

cast_m = vec | n_rays = 10000 | DEM points: 10000
Cast time: 2.28 s | Timeout: False

cast_m = seq | n_rays = 10000 | DEM points: 10000
Cast time: 3.28 s | Timeout: False

cast_m = vec | n_rays = 1 | DEM points: 40000
Cast time: 0.59 s | Timeout: False

cast_m = seq | n_rays = 1 | DEM points: 40000
Cast time: 0.21 s | Timeout: False

cast_m = vec | n_rays = 10 | DEM points: 40000
Cast time: 0.64 s | Timeout: False

cast_m = seq | n_rays = 10 | DEM points: 40000
Cast time: 0.13 s | Timeout: False

cast_m = vec | n_rays = 100 | DEM points: 40000
Cast time: 0.70 s | Timeout: False

cast_m = seq | n_rays = 100 | DEM points: 40000
Cast time: 0.20 s | Timeout: False

cast_m = vec | n_rays = 1000 | DEM points: 40000
Cast time: 1.05 s | Timeout: False

cast_m = seq | n_rays = 1000 | DEM points: 40000
Cast time: 0.96 s | Timeout: False

cast_m = vec | n_rays = 10000 | DEM points: 40000
Cast time: 3.98 s | Timeout: False

cast_m = seq | n_rays = 10000 | DEM points: 40000
Cast time: 9.14 s | Timeout: False

cast_m = vec | n_rays = 1 | DEM points: 250000
Cast time: 3.37 s | Timeout: False

cast_m = seq | n_rays = 1 | DEM points: 250000
Cast time: 0.70 s | Timeout: False

cast_m = vec | n_rays = 10 | DEM points: 250000
Cast time: 3.42 s | Timeout: False

cast_m = seq | n_rays = 10 | DEM points: 250000
Cast time: 0.16 s | Timeout: False

cast_m = vec | n_rays = 100 | DEM points: 250000
Cast time: 3.57 s | Timeout: False

cast_m = seq | n_rays = 100 | DEM points: 250000
Cast time: 0.64 s | Timeout: False

cast_m = vec | n_rays = 1000 | DEM points: 250000
Cast time: 4.52 s | Timeout: False

cast_m = seq | n_rays = 1000 | DEM points: 250000
Cast time: 5.28 s | Timeout: False

cast_m = vec | n_rays = 10000 | DEM points: 250000
Cast time: 12.05 s | Timeout: False

cast_m = seq | n_rays = 10000 | DEM points: 250000
Cast time: 52.02 s | Timeout: False

cast_m = vec | n_rays = 1 | DEM points: 1000000
Cast time: 17.07 s | Timeout: False

cast_m = seq | n_rays = 1 | DEM points: 1000000
Cast time: 2.30 s | Timeout: False

cast_m = vec | n_rays = 10 | DEM points: 1000000
Cast time: 18.11 s | Timeout: False

cast_m = seq | n_rays = 10 | DEM points: 1000000
Cast time: 0.29 s | Timeout: False

cast_m = vec | n_rays = 100 | DEM points: 1000000
Cast time: 17.95 s | Timeout: False

cast_m = seq | n_rays = 100 | DEM points: 1000000
Cast time: 2.07 s | Timeout: False

cast_m = vec | n_rays = 1000 | DEM points: 1000000
Cast time: 19.30 s | Timeout: False

cast_m = seq | n_rays = 1000 | DEM points: 1000000
Cast time: 19.79 s | Timeout: False

cast_m = vec | n_rays = 10000 | DEM points: 1000000
Cast time: 35.79 s | Timeout: False

cast_m = seq | n_rays = 10000 | DEM points: 1000000
Cast time: 197.09 s | Timeout: False
```